論文有提及，中文的ABSA資料過少，因此需要翻譯英文的相關資料

這個程式就是翻譯的程式，是套用google translate的API

需要注意的點是一次翻譯太多會出問題(到後面他就不會翻了)，所以要分批/分時翻譯

In [1]:
#pip install googletrans==3.1.0a0
#靠北ip被鎖了
import googletrans
translator = googletrans.Translator()

In [1]:
with open('./data/ABSA_Eng/Restaurant_train.txt', 'r') as file:
    resturant_train = file.readlines()
with open('./data/ABSA_Eng/Restaurant_test.txt', 'r') as file:
    resturant_test = file.readlines()
with open('./data/ABSA_Eng/Laptop_train.txt', 'r') as file:
    laptop_train = file.readlines()  
with open('./data/ABSA_Eng/Laptop_test.txt', 'r') as file:
    laptop_test = file.readlines()  
#print(translator.translate('this is a book.', src='en', dest='zh-tw').text)

In [3]:
rest_train_sent, rest_test_sent = [], []
laptop_train_sent, laptop_test_sent = [], []
for i in range(len(resturant_train)):
    if (i+1)%3==0:
        rest_train_sent.append(int(resturant_train[i]))
        
for i in range(len(resturant_test)):
    if (i+1)%3==0:
        rest_test_sent.append(int(resturant_test[i]))
        
for i in range(len(laptop_train)):
    if (i+1)%3==0:
        laptop_train_sent.append(int(laptop_train[i]))
        
for i in range(len(laptop_test)):
    if (i+1)%3==0:
        laptop_test_sent.append(int(laptop_test[i]))

In [4]:
from collections import Counter
print(Counter(rest_train_sent), Counter(rest_test_sent))
print(Counter(laptop_train_sent), Counter(laptop_test_sent))

Counter({1: 2164, -1: 807, 0: 637}) Counter({1: 728, 0: 196, -1: 196})
Counter({1: 994, -1: 870, 0: 464}) Counter({1: 341, 0: 169, -1: 128})


In [5]:
rest_train_sentence, rest_test_sentence = [], []
rest_train_target, rest_test_target = [], []
laptop_train_sentence, laptop_test_sentence = [], []
laptop_train_target, laptop_test_target = [], []
for i in range(len(resturant_train)):
    if (i)%3==0:
        rest_train_sentence.append(resturant_train[i].replace('$T$', resturant_train[i+1][:-1])[:-1])
        rest_train_target.append(resturant_train[i+1][:-1])
        
for i in range(len(resturant_test)):
    if (i)%3==0:
        rest_test_sentence.append(resturant_test[i].replace('$T$', resturant_test[i+1][:-1])[:-1])
        rest_test_target.append(resturant_test[i+1][:-1])
        
for i in range(len(laptop_train)):
    if (i)%3==0:
        laptop_train_sentence.append(laptop_train[i].replace('$T$', laptop_train[i+1][:-1])[:-1])
        laptop_train_target.append(laptop_train[i+1][:-1])
        
for i in range(len(laptop_test)):
    if (i)%3==0:
        laptop_test_sentence.append(laptop_test[i].replace('$T$', laptop_test[i+1][:-1])[:-1])
        laptop_test_target.append(laptop_test[i+1][:-1])

## Direct convert into .json file

In [20]:
rest_train, rest_test, lap_train, lap_test = [], [], [], []
sentid = {-1:'負向', 0:'中立', 1:'正向'}
for i in range(len(rest_train_sentence)):
    tmp_dict = {'sentence':rest_train_sentence[i], 'target':rest_train_target[i], 'sentiment':sentid[rest_train_sent[i]]}
    rest_train.append(tmp_dict)

for i in range(len(rest_test_sentence)):
    tmp_dict = {'sentence':rest_test_sentence[i], 'target':rest_test_target[i], 'sentiment':sentid[rest_test_sent[i]]}
    rest_test.append(tmp_dict)

for i in range(len(laptop_train_sentence)):
    tmp_dict = {'sentence':laptop_train_sentence[i], 'target':laptop_train_target[i], 'sentiment':sentid[laptop_train_sent[i]]}
    lap_train.append(tmp_dict)

for i in range(len(laptop_test_sentence)):
    tmp_dict = {'sentence':laptop_test_sentence[i], 'target':laptop_test_target[i], 'sentiment':sentid[laptop_test_sent[i]]}
    lap_test.append(tmp_dict)

In [23]:
import json
with open('./data/ABSA_Eng/restaurant_train.json', 'w', encoding='utf8') as file:
    count=0
    for d in rest_train:
        ret = json.dumps(d, ensure_ascii=False)
        file.write(ret)
        file.write('\n')
        count+=1
    print('restaurant train 共寫了{}個份資料'.format(count))
    
with open('./data/ABSA_Eng/restaurant_test.json', 'w', encoding='utf8') as file:
    count=0
    for d in rest_test:
        ret = json.dumps(d, ensure_ascii=False)
        file.write(ret)
        file.write('\n')
        count+=1
    print('restaurant test 共寫了{}個份資料'.format(count))

with open('./data/ABSA_Eng/laptop_train.json', 'w', encoding='utf8') as file:
    count=0
    for d in lap_train:
        ret = json.dumps(d, ensure_ascii=False)
        file.write(ret)
        file.write('\n')
        count+=1
    print('laptop train 共寫了{}個份資料'.format(count))

with open('./data/ABSA_Eng/laptop_test.json', 'w', encoding='utf8') as file:
    count=0
    for d in lap_test:
        ret = json.dumps(d, ensure_ascii=False)
        file.write(ret)
        file.write('\n')
        count+=1
    print('laptop test 共寫了{}個份資料'.format(count))


restaurant train 共寫了3608個份資料
restaurant test 共寫了1120個份資料
laptop train 共寫了2328個份資料
laptop test 共寫了638個份資料


## Translate into Chinese(zh)

In [11]:
import time
starttime = time.time()
resturant_data = []
startendpos = '※'
wrong = []
for i in range(274, 277):
    tmp = {'sentence':'', 'target':'', 'sentiment':0}    
    tmp['sentence'] = translator.translate(resturant[i*3][:-1].replace('$T$', startendpos+resturant[i*3+1][:-1]+startendpos), src='en', dest='zh-CN').text
    flag = False
    for c in tmp['sentence']:
        if c == startendpos:
            flag = not flag
        elif flag:
            tmp['target']+=c
    tmp['sentence'] = tmp['sentence'].replace(startendpos, '')
    tmp['sentiment'] = int(resturant[i*3+2][:-1])    
    if tmp['target'] not in tmp['sentence']:
        wrong.append(i)
    if i%100==0:
        print(i, 'spend', int(time.time()-starttime))
    resturant_data.append(tmp)

# read translated data

In [6]:
import json
restaurant, laptop = [], []
with open('./data/translate/resturant.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    restaurant.append(json.loads(d))
with open('./data/translate/laptop.json', 'r', encoding='utf8') as file:
    data = file.readlines()
for d in data:
    laptop.append(json.loads(d))

In [12]:
sentid = {-1:'負向', 0:'中立', 1:'正向'}

In [13]:
#find target not in sentence
restaurant_true = []
laptop_true = []
for data in restaurant:
    if data['target'] in data['sentence']:
        data['sentiment'] = sentid[data['sentiment']]
        restaurant_true.append(data)
print('restaurant size:', len(restaurant_true))
for data in laptop:
    if data['target'] in data['sentence']:
        data['sentiment'] = sentid[data['sentiment']]
        laptop_true.append(data)
print('laptop size:', len(laptop_true))

restaurant size: 1908
laptop size: 1220


In [15]:
with open('./data/translate/filt_restaurant.json', 'w', encoding='utf8') as file:
    count=0
    for d in restaurant_true:
        ret = json.dumps(d, ensure_ascii=False)
        file.write(ret)
        file.write('\n')
        count+=1
    print('restaurant 共寫了{}個份資料'.format(count))
    
with open('./data/translate/filt_laptop.json', 'w', encoding='utf8') as file:
    count=0
    for d in laptop_true:
        ret = json.dumps(d, ensure_ascii=False)
        file.write(ret)
        file.write('\n')
        count+=1
    print('laptop 共寫了{}個份資料'.format(count))

restaurant 共寫了1908個份資料
laptop 共寫了1220個份資料
